In [ ]:
import numpy as np
import pandas as pd
import pycountry
from datetime import datetime
import os

import requests
from bs4 import BeautifulSoup
import csv
import re

In [ ]:
#collect list of Top100 airlines (source: https://www.flightsfrom.com/top-100-airlines)


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) /Chrome/88.0.4324.104 Safari/537.36'}


tmplist=[]

counter=0
response=requests.get(f'https://www.flightsfrom.com/top-100-airlines',headers=headers)
soup=BeautifulSoup(response.text, 'html.parser')

tmplist=soup.find_all('span', attrs = {'class': 'hometoplist-first'})

top100_airlines=[]

for tmp in tmplist:
    top100_airlines.append(tmp.get_text().strip())
    

In [ ]:
#iterate through all flight data files and collect grouped operator/owner data
import pandas as pd

import os

operator_data=pd.read_csv('aircraftDatabase-2021-07.csv')
operator_data.operator.fillna(operator_data.owner, inplace=True)
operator_data_filtered=operator_data.loc[:,['icao24','operator']]

csv_data=pd.read_csv('flightlist_20210601_20210630.csv')
csv_data2=csv_data[['callsign','icao24','origin','destination','firstseen','lastseen']]
flight_data_consolidated=csv_data2.merge(operator_data_filtered, on='icao24', how='left')
flight_data_grouped_total=flight_data_consolidated.groupby('operator').agg('count').sort_values('callsign', ascending=False)


counter=1
directory = r'/Users/moritz/Desktop/covid-vaccination-flight-levels/flight_data'
for entry in os.scandir(directory):
    if entry.path.endswith(".csv") and entry.is_file():
        print (counter)
        csv_data=pd.read_csv(entry)
        csv_data2=csv_data[['callsign','icao24','origin','destination','firstseen','lastseen']]
        flight_data_consolidated=csv_data2.merge(operator_data_filtered, on='icao24', how='left')
        flight_data_grouped=flight_data_consolidated.groupby('operator').agg('count').sort_values('callsign', ascending=False)
        flight_data_grouped_total=pd.concat([flight_data_grouped_total,flight_data_grouped],axis=0, sort=False)
        print ('\n')
        print (entry)
        counter+=1

flight_data_grouped_total=flight_data_grouped_total.groupby('operator').agg('sum').sort_values('callsign', ascending=False)
flight_data_grouped_total.to_csv('grouped_operators.csv')        


In [ ]:
#Fuzzy match operator data with Top100 airline list

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

airline_mapping=pd.DataFrame(columns=['operator','airline_match','fuzzy_rank'])

for operator_ in list(flight_data_grouped_total.index):
    dict_={}
    for airline in top100_airlines:
        dict_[airline]=fuzz.ratio(operator_,airline)
    maximum_fuzzy_match=max(dict_, key=lambda key: dict_[key])
    maximum_fuzzy_value=max(dict_.values())
    airline_mapping.loc[len(airline_mapping.index)]=[operator_,maximum_fuzzy_match,maximum_fuzzy_value]


airline_mapping.to_csv('airline_mapping.csv')

#use fuzzy-matched data and manually adjust/ correct (overall, reduced # of relevant flights from #57297620 to 32051497)
airline_mapping_corrected=pd.read_csv('airline_mapping_manual.csv')


flight_data_grouped_corrected=flight_data_grouped_total.merge(airline_mapping_corrected, on='operator', how='left')

relevant_operators=flight_data_grouped_corrected[~flight_data_grouped_corrected['airline_match'].isnull()]['operator']


In [ ]:
#concatenate all monthly flight data files into one long table
import pandas as pd

import os

flight_data=pd.read_csv('flightlist_20210601_20210630.csv')
flight_data_consolidated=flight_data.merge(operator_data_filtered, on='icao24', how='left')        
flight_data_consolidated=flight_data_consolidated[flight_data_consolidated['operator'].isin(relevant_operators)]
flight_data_consolidated2=flight_data_consolidated[['callsign','icao24','origin','destination','firstseen','lastseen']] 



counter=1
directory = r'/Users/moritz/Desktop/covid-vaccination-flight-levels/flight_data'
for entry in os.scandir(directory):
    if entry.path.endswith(".csv") and entry.is_file():
        print (counter)
        flight_data=pd.read_csv(entry)
        flight_data_com=flight_data.merge(operator_data_filtered, on='icao24', how='left')        
        flight_data_com=flight_data_com[flight_data_com['operator'].isin(relevant_operators)]
        flight_data_com=flight_data_com[['callsign','icao24','origin','destination','firstseen','lastseen']]  
        
        
        flight_data_consolidated2=pd.concat([flight_data_consolidated2,flight_data_com],axis=0, sort=False)


        print ('\n')
        print (entry)
        counter+=1

flight_data_consolidated2.to_csv(f'flight_data_cons_1.csv')


In [ ]:

#read flight data (total)
flight_data=pd.read_csv('flight_data_cons_1.csv')


#change date in flight data from string to datetime object
def datecheck(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S+00:00')

flight_data['firstseen_num']=flight_data['firstseen'].apply(datecheck)
flight_data['lastseen_num']=flight_data['lastseen'].apply(datecheck)
flight_data['date_num']=flight_data['lastseen_num'].apply(datetime.date)

#--------------------

#read airport mapping data mapping airport codes to countries. Map countries to 3 digit ISO codes
airport_mapping=pd.read_csv('airport-codes_csv.csv')

def country_mapping(x):
    try:
        y=pycountry.countries.get(alpha_2=x).alpha_3
    except:
        y=''
    return y

airport_mapping['iso_country_3dig']=airport_mapping['iso_country'].apply(country_mapping)

#--------------------
#Merge flight data and airport to country mapping

flight_data_country=flight_data.merge(airport_mapping, how='left', left_on='origin', right_on='ident')

flight_data_country=flight_data_country[['callsign','icao24','origin','destination','firstseen_num','lastseen_num','date_num','ident','name','continent','iso_country','iso_country_3dig']]

#--------------------
#read vaccination data over time per country
vaccination_data=pd.read_csv('owid-covid-data.csv')


#filter out irrelevant columns and lines, and add vaccination percentage column
filtered_vaccination_data=vaccination_data[['iso_code','location','date','people_fully_vaccinated_per_hundred','population']]
filtered_vaccination_data=filtered_vaccination_data.loc[filtered_vaccination_data['date']>'2020-12-15']

#add correctly formatted date column to vaccination dataset
def datecheck_vac(x):
    return datetime.strptime(x, '%Y-%m-%d').date()

filtered_vaccination_data['date_num']=filtered_vaccination_data['date'].apply(datecheck_vac)

#--------------------
#Merge vaccination data into flight data

flight_data_completed=flight_data_country.merge(filtered_vaccination_data, how='left', left_on=['iso_country_3dig','date_num'], right_on=['iso_code','date_num'])

flight_data_completed.to_csv('final_flight_table.csv')

